<a href="https://colab.research.google.com/github/atharva434/Safar-chatbot/blob/master/predict/static/travel_chatbot_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json 
import matplotlib.pyplot as plt 

In [ ]:
path = '/content/New_intents.json'

with open(path) as file : 
  data = json.load(file)


training_sentences = [] 
training_labels = [] 
labels = [] 
responses = [] 

for intent in data['intents'] : 
  for pattern in intent['patterns']:
    training_sentences.append(pattern) 
    training_labels.append(intent['tag'])
  responses.append(intent['responses']) 

  labels.append(intent['tag'])

num_classes = len(labels)


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(training_labels)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)

[[0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0.

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
vocab_size = 1000 
embedding_dim = 32 
max_len = 20 
oov_token = "<OOV>"
tokenizer = Tokenizer(num_words = vocab_size , oov_token = oov_token , lower = True , filters='!"#$%&()*+,-./:;<=>@[\\]^_`{|}~\t\n' , split = ' ' )

In [ ]:
tokenizer.(list(training_sentences)) 

In [ ]:
list_tokenized_train = tokenizer.texts_to_sequences(training_sentences)

In [ ]:
max_len = 10 
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', padding = 'post', maxlen=max_len)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, onehot_encoded, test_size = 0.20, shuffle = True, stratify=onehot_encoded)

In [ ]:
print("Train Dimensions : " ,X_train.shape)
print("Test Dimensions : " ,X_test.shape)


Train Dimensions :  (55, 10)
Test Dimensions :  (14, 10)


In [ ]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.models import Model
from keras.layers import Bidirectional, GlobalMaxPool1D




In [ ]:
this is a fantastic fucking movie. I fucking hate it

In [ ]:
#input layer 
inp = Input(shape = (max_len , ))
max_features = vocab_size
#embedding layer
embed = embedding_dim
x = Embedding(max_features , embed , mask_zero = True)(inp)
#lstm layer 
x = LSTM(64, return_sequences=True,name='lstm_layer')(x)
#max pooling for reducing dimensions 
x = GlobalMaxPool1D()(x)
#dropout layer
x = Dropout(0.2)(x)
#Dense Layer 
x = Dense(32, activation="relu")(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.2)(x)
#Output Layer
x = Dense(6, activation="softmax")(x)




In [ ]:
model = Model(inputs=inp, outputs=x)
model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10)]              0         
                                                                 
 embedding (Embedding)       (None, 10, 32)            32000     
                                                                 
 lstm_layer (LSTM)           (None, 10, 64)            24832     
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                             

In [ ]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

In [ ]:
callback = EarlyStopping(monitor = 'val_loss' , patience = 10 , mode = 'min' )

In [ ]:
filepath = '/content/model.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode = 'max')

callback_list = [checkpoint ]

In [ ]:
epochs = 500
history=model.fit(X_train, y_train, batch_size=8, epochs=epochs,validation_data=(X_test,y_test) , callbacks = callback_list)

Epoch 1/500
6/7 [========================>.....] - ETA: 0s - loss: 1.7907 - accuracy: 0.2083 
Epoch 1: val_accuracy improved from -inf to 0.35714, saving model to /content/model.h5
7/7 [==============================] - 7s 275ms/step - loss: 1.7904 - accuracy: 0.2182 - val_loss: 1.7875 - val_accuracy: 0.3571
Epoch 2/500
6/7 [========================>.....] - ETA: 0s - loss: 1.7840 - accuracy: 0.3125
Epoch 2: val_accuracy did not improve from 0.35714
7/7 [==============================] - 0s 16ms/step - loss: 1.7832 - accuracy: 0.3273 - val_loss: 1.7830 - val_accuracy: 0.3571
Epoch 3/500
6/7 [========================>.....] - ETA: 0s - loss: 1.7772 - accuracy: 0.3333
Epoch 3: val_accuracy did not improve from 0.35714
7/7 [==============================] - 0s 16ms/step - loss: 1.7770 - accuracy: 0.3455 - val_loss: 1.7757 - val_accuracy: 0.2857
Epoch 4/500
6/7 [========================>.....] - ETA: 0s - loss: 1.7693 - accuracy: 0.3542
Epoch 4: val_accuracy improved from 0.35714 to 0.4285

In [ ]:
import pickle

# to save the fitted tokenizer
with open('/content/token.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('/content/encoder.pickle', 'wb') as ecn_file:
    pickle.dump(label_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
from tensorflow import keras
import random
import numpy as np
import pickle

In [ ]:
def chat():
    # load trained model
    chat_model = keras.models.load_model('/content/model.h5')

    # load tokenizer object
    with open('/content/token.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('/content/encoder.pickle', 'rb') as enc:
        onehot_encoded = pickle.load(enc)

    # parameters
    max_len = 10
    
    while True:
        
        inp = input()
        if inp.lower() == "quit":
            break

        result = chat_model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = label_encoder.inverse_transform([np.argmax(result)])
        

        for i in data['intents']:
            if i['tag'] == tag:
                print("ChatBot:", np.random.choice(i['responses']))
chat()

what about disabled
ChatBot: You can pay with any nationalised or private bank credit card, or you can use paytm or google pay to pay with your upi.
how to pay for ticket
ChatBot: You can pay with any nationalised or private bank credit card, or you can use paytm or google pay to pay with your upi.
where to scan the qr code
ChatBot: 
 QR code will be scanned at the monument entry gate and status of verification will be displayed on the lcd 


KeyboardInterrupt: ignored